# Step04: Apply double-logistic regression
We used here the same vegetation indices as in Predfine<br>
- Input: Timeseries of vegetaion indices, planting_date and some paramters<br>
- Output: dataframe file of interpolated vegetation indice   

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.optimize import curve_fit
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

## Inputs

In [2]:
vegindices_ts_file = '/home/s1093765/yield/vegetation_indices_timeseries.gzip'
outfile = '/home/s1093765/yield/vegetation_indices_timeseries.inetrpolated.gzip'
yield_file = '/home/s1093765/yield/yield_polygons.csv'
output_param_file = '/home/s1093765/yield/paramters.csv'


plantingDate_var = 'Planting_date'
datetime_var = 'datetime'
id_var = 'id2'
yield_variables = ['yield_median', 'yield_mean', 'yield_std', 'yield_num']



num_paramters = 7
dday_min=-60
dday_max=300
range_days_min = 250
num_samples_min = 20
r_value_min = 0.98
std_k = 2 # for paramters bounds (mean+/- (std_k * std))


vkeys = ['NDVI', 'EVI2',
       'MSAVI', 'CVI', 'NDWI', 'NDDI', 'GNDVI', 'GVMI', 'PVI', 'SLAVI', 'GVI',
       'MCARI', 'VSDI', 'fapar', 'LAI', 'Cab']
vstat = ['median', 'mean', 'q10', 'q90']

temporal_resapling='3D' #median resapling over #days to avoid redandancy if we hape multi images within short period
bins='1D' #output temporal frequency: daily:1D, 2-daily:2D, ..

## Read vegetation indices and file of planting dates

In [3]:
df = pd.read_parquet(vegindices_ts_file)[:1000]
df[datetime_var] = pd.to_datetime(df[datetime_var])
df.head(2)

,Unnamed: 0,"('NDVI', 'mean')","('NDVI', 'median')","('NDVI', 'std')","('NDVI', 'q10')","('NDVI', 'q25')","('NDVI', 'q75')","('NDVI', 'q90')","('EVI2', 'mean')","('EVI2', 'median')",...,"('Cab', 'mean')","('Cab', 'median')","('Cab', 'std')","('Cab', 'q10')","('Cab', 'q25')","('Cab', 'q75')","('Cab', 'q90')",datetime,id2,num_pixs
0,0,0.212710,0.188566,0.075381,0.171281,0.178694,0.218957,0.284676,0.148154,0.139368,...,0.876044,0.647051,0.688763,0.480410,0.553662,0.84229,1.417008,2019-04-08 16:35:32,2019__98.2818W_42.2815N,3282
1,1,0.261607,0.250802,0.047065,0.231345,0.240909,0.267648,0.290229,0.222100,0.214921,...,0.534302,0.423106,0.443637,0.283815,0.341399,0.56753,0.800096,2019-05-03 16:32:18,2019__98.2818W_42.2815N,3282


In [4]:
df_yield = pd.read_csv(yield_file)[[plantingDate_var, id_var]+yield_variables]

In [5]:
df_yield[plantingDate_var] = pd.to_datetime(df_yield[plantingDate_var])
df_yield.head(2)

,Planting_date,id2,yield_median,yield_mean,yield_std,yield_num
0,2019-04-21,2019__98.2818W_42.2815N,255.40,255.40,0.0,1.0
1,2019-04-11,2019__89.63516W_36.78634N,228.65,228.65,0.0,1.0


In [6]:
#just to rename columns (remove '(, )')
def rename_variable(var=None):
    out = var
    for c in ['(', ')', ',', "'"]:
        out = out.replace(c, '')
    out = out.replace(' ', '_')
    return out
for k in df.keys():
    df = df.rename(columns={k: rename_variable(k)})

In [7]:
df.head(2)

,Unnamed:_0,NDVI_mean,NDVI_median,NDVI_std,NDVI_q10,NDVI_q25,NDVI_q75,NDVI_q90,EVI2_mean,EVI2_median,...,Cab_mean,Cab_median,Cab_std,Cab_q10,Cab_q25,Cab_q75,Cab_q90,datetime,id2,num_pixs
0,0,0.212710,0.188566,0.075381,0.171281,0.178694,0.218957,0.284676,0.148154,0.139368,...,0.876044,0.647051,0.688763,0.480410,0.553662,0.84229,1.417008,2019-04-08 16:35:32,2019__98.2818W_42.2815N,3282
1,1,0.261607,0.250802,0.047065,0.231345,0.240909,0.267648,0.290229,0.222100,0.214921,...,0.534302,0.423106,0.443637,0.283815,0.341399,0.56753,0.800096,2019-05-03 16:32:18,2019__98.2818W_42.2815N,3282


In [8]:
df = pd.merge(df, df_yield, on=id_var)

In [9]:
df.head(2)

,Unnamed:_0,NDVI_mean,NDVI_median,NDVI_std,NDVI_q10,NDVI_q25,NDVI_q75,NDVI_q90,EVI2_mean,EVI2_median,...,Cab_q75,Cab_q90,datetime,id2,num_pixs,Planting_date,yield_median,yield_mean,yield_std,yield_num
0,0,0.212710,0.188566,0.075381,0.171281,0.178694,0.218957,0.284676,0.148154,0.139368,...,0.84229,1.417008,2019-04-08 16:35:32,2019__98.2818W_42.2815N,3282,2019-04-21,255.4,255.4,0.0,1.0
1,1,0.261607,0.250802,0.047065,0.231345,0.240909,0.267648,0.290229,0.222100,0.214921,...,0.56753,0.800096,2019-05-03 16:32:18,2019__98.2818W_42.2815N,3282,2019-04-21,255.4,255.4,0.0,1.0


## Regression functions

In [10]:
def double_logistic(x, vb, ve, k, c, p, d, q):
    return vb + (k/(1+np.exp(-c*(x-p)))) - ((k+vb+ve)/(1+np.exp(d*(x-q))))


def uniform_distr(df=None, resample=None, bins=None, planting_date = None, 
                  dday_min=None, dday_max=None, parameters0=None, bounds=[-np.Inf, np.Inf], method='lm'):

    dfo = df.resample(resample).median()
    dfo = dfo.resample(bins).median()
    
    if isinstance(planting_date, str):
        planting_date = pd.to_datetime([planting_date]).values[0]
    x = (dfo.index.values - planting_date).astype('timedelta64[D]').astype('float')    
   
    y = dfo.values[:, 0]
    w = np.where((x>=dday_min) & (x<=dday_max) & (~np.isnan(y)) & (~np.isinf(y)))
    
    x_norm = (x-dday_min)/(dday_max-dday_min)
   
    out = None
    try:
        popt, _ = curve_fit(double_logistic, x_norm[w], y[w], 
                                           check_finite=True,method=method,
                            p0=parameters0, maxfev=6000,  bounds=bounds)
        slope, intercept, r_value, p_value, std_err = stats.linregress(y[w], double_logistic(x_norm[w], *popt))

        
        dfo[df.keys()[0]] = double_logistic(x_norm, *popt)
        out = {'dataframe': dfo, 'r_value': r_value, 'std_error': std_err, 'parameters': popt}
    except:
        pass
    return out


def initial_parameters(df = None, num_min = None, rangedays_min = None, var=None, parameters0=None, 
                       dday_min=None, dday_max=None, 
                       datetime_var= None, plantingDate_var=None):
    n_samples = len(df[id_var].unique())
    ddays = np.arange(dday_min, dday_max+1)
    norm_ddays = (ddays-dday_min) / (dday_max-dday_min)
    #values = np.ones((n_samples, ddays.shape[0]), 'f4') * np.nan
    dct1 = {}
    dct1['parameter'] = np.ones((n_samples, 7), 'f4') * np.nan
    dct1['r_value'] = np.ones((n_samples), 'f4') * np.nan
    dct1['std_err'] = np.ones((n_samples), 'f4') * np.nan
    dct1['range_days'] = np.zeros((n_samples), int)
    dct1['num'] = np.zeros((n_samples), 'int') 
    
    stats = pd.DataFrame()

    i = 0
    n = 0
    
    r2_max = -1
    p0 = None
    for id2 in df[id_var].unique():
            df1 = df[df[id_var] ==id2]
            df1.index = df1[datetime_var]
            rangedays =  (df1.index.values.max() - df1.index.values.min()).astype('timedelta64[D]').astype('int')
            if rangedays>=rangedays_min and df1.shape[0]>num_min:
                dct = uniform_distr(df=df1[[var]], resample='6D', bins='1D', planting_date=df1[plantingDate_var].values[0], parameters0=None,
                                     dday_min=dday_min, dday_max=dday_max)
                
             
                if not dct is None:
                        #paramters0 = dct['parameters']
                        dct1['r_value'][i] = dct['r_value']
                        dct1['std_err'][i] = dct['std_error']
                        dct1['num'][i] = df1.shape[0]
                        dct1['range_days'][i] = rangedays
                        
                        #dct['values'][i, :] = double_logistic(norm_ddays, *dct['parameters'])
                        dct1['parameter'][i, :] = np.array(dct['parameters'])
                        n +=1
     
            i +=1
    print(var, 'success fitting ratio:', 100*n/i, '%')
   
    return dct1


## Compute initial paramters

In [11]:
dfp = pd.DataFrame()
for var0 in vkeys:
        var = var0+'_median'
        dct = initial_parameters(df = df, var = var,  parameters0=None, 
                                 num_min = num_samples_min, rangedays_min = range_days_min,
                                 dday_max=dday_max, dday_min=dday_min,
                                datetime_var=datetime_var, plantingDate_var=plantingDate_var)

        dfp1 = pd.DataFrame()
        for k in dct.keys():
            if len(dct[k].shape) ==1:
                dfp1[k] = dct[k]
            elif len(dct[k].shape) ==2:
                    for i in range(dct[k].shape[1]):
                        dfp1[k+str(i+1)] = dct[k][:, i]

        dfp1 = dfp1.dropna()
        dfp1['variable'] = var0
        dfp = pd.concat([dfp, dfp1])
        #dfp.to_csv(outfile)
    #else:
     #   print('exist: '+outfile)

NDVI_median success fitting ratio: 72.22222222222223 %
EVI2_median success fitting ratio: 75.0 %
MSAVI_median success fitting ratio: 63.888888888888886 %
CVI_median success fitting ratio: 69.44444444444444 %
NDWI_median success fitting ratio: 75.0 %
NDDI_median success fitting ratio: 52.77777777777778 %
GNDVI_median success fitting ratio: 75.0 %
GVMI_median success fitting ratio: 75.0 %
PVI_median success fitting ratio: 75.0 %
SLAVI_median success fitting ratio: 72.22222222222223 %
GVI_median success fitting ratio: 75.0 %
MCARI_median success fitting ratio: 63.888888888888886 %
VSDI_median success fitting ratio: 63.888888888888886 %
fapar_median success fitting ratio: 75.0 %
LAI_median success fitting ratio: 75.0 %
Cab_median success fitting ratio: 75.0 %


In [12]:
dfp.head(10)

,parameter1,parameter2,parameter3,parameter4,parameter5,parameter6,parameter7,r_value,std_err,range_days,num,variable
1,0.295754,-0.222551,0.565218,210.756516,0.283798,-76.963303,0.541068,0.984411,0.034629,280,28,NDVI
2,0.517097,-0.196685,382.961304,15.092113,0.341214,-15.008905,0.341568,0.948432,0.067223,349,25,NDVI
3,0.268598,0.104705,183.548965,11.121944,0.297500,-11.026426,0.299095,0.980286,0.048422,275,23,NDVI
5,0.375576,-0.213598,115.863853,15.616585,0.331105,-15.381693,0.332314,0.967119,0.052437,279,28,NDVI
7,0.640704,-0.088658,287.611328,12.196549,0.264364,-12.091354,0.264710,0.969545,0.047490,304,29,NDVI
8,1.664839,-0.182308,531.491150,12.849129,0.250469,-12.761532,0.250180,0.946086,0.058976,349,31,NDVI
9,0.253050,-0.229070,0.625364,109.718292,0.246776,-46.779331,0.486620,0.992472,0.024811,325,32,NDVI
10,0.236126,-0.243698,0.656588,53.624115,0.221197,-57.245049,0.430917,0.991448,0.026979,304,27,NDVI
11,0.820293,-0.341522,204.481873,16.976688,0.293049,-16.803635,0.293108,0.868991,0.077230,349,44,NDVI
12,0.709501,-0.101368,241.007141,12.241755,0.270720,-12.108603,0.271129,0.954009,0.058378,325,31,NDVI


In [13]:
param_keys = ['parameter'+str(i+1) for i in range(num_paramters)]

In [14]:
dfps = pd.DataFrame()
for var in dfp['variable'].unique():
        dfp1 = dfp[(dfp['variable'] == var) & 
                   (dfp['r_value']>=r_value_min) &
                  (dfp['range_days']>range_days_min)]
        df1 = pd.DataFrame()
        df1['Parameter'] = np.arange(1, len(param_keys)+1)
        df1['Variable'] = var
        df1['Median'] = np.median(dfp1[param_keys].values, axis=0)
        df1['Mean'] = np.mean(dfp1[param_keys].values, axis=0)
        df1['Std'] = np.std(dfp1[param_keys].values, axis=0)
        df1['Num'] = np.sum(~np.isnan(dfp1[param_keys].values), axis=0)
        df1['R_value'] = np.median(dfp1['r_value'].values, axis=0)
        df1['Std_err'] = np.median(dfp1['std_err'].values, axis=0)
        dfps = pd.concat([dfps, df1])
dfps

,Parameter,Variable,Median,Mean,Std,Num,R_value,Std_err
0,1,NDVI,0.269426,0.267378,0.026638,10,0.996418,0.019082
1,2,NDVI,-0.216220,-0.178531,0.099256,10,0.996418,0.019082
2,3,NDVI,0.632183,18.916508,54.877495,10,0.996418,0.019082
3,4,NDVI,70.525024,113.695229,88.686333,10,0.996418,0.019082
4,5,NDVI,0.279551,0.267372,0.028394,10,0.996418,0.019082
...,...,...,...,...,...,...,...,...
2,3,Cab,-36.577419,-20.335608,36.022457,4,0.998896,0.009237
3,4,Cab,45.475853,-70.170677,204.497559,4,0.998896,0.009237
4,5,Cab,0.353647,0.360698,0.096666,4,0.998896,0.009237
5,6,Cab,-3.544842,34.104759,108.208916,4,0.998896,0.009237


In [15]:
#save initial paramters
dfps.to_csv(output_param_file)

## Apply regression to create daily time-series

In [16]:
dfs = []
i = 1
for id2 in df[id_var].unique():
            df1 = df[df[id_var].str.contains(id2)]
            df1 = df1.sort_values(by=[datetime_var])
            df1.index = df1[datetime_var]
            planting_date = df1[df1[id_var]==id2][plantingDate_var].values[0]
            dfOut1 = pd.DataFrame()
            for v in vkeys:
                parameters0 = None
                bounds = [-np.Inf, np.Inf]
                dfps1 = dfps[dfps.Variable == v]
                if dfps1.shape[0]:
                    dfps1 = dfps1.sort_values(by=['Parameter'])
                    parameters0 = dfps1['Median'].values
                    bounds = np.array([[dfps1['Median'].values - (std_k * dfps1['Std'].values)], 
                                       [dfps1['Median'].values + (std_k * dfps1['Std'].values)]])[:,0,:]
                for s in vstat:
                    var = v+'_'+s
                    out =  uniform_distr(df=df1[[var]], resample=temporal_resapling, 
                                         bins=bins, planting_date = planting_date, 
                                         parameters0=parameters0,
                                      dday_min=dday_min, dday_max=dday_max,
                                         bounds=bounds, method='dogbox')
                    if out is not None: 
                            df2 = out['dataframe']
                            dfOut1 = pd.concat([dfOut1, df2], axis=1)



            if(dfOut1 is not  None and dfOut1.shape[0]>0):
                for k1 in [id_var, plantingDate_var]+yield_variables:
                        dfOut1[k1] = df1[k1].values[0]
                        dfs.append(dfOut1)

            dfOut1 = None
            df1 = None
            i+=1

In [34]:
dfs = pd.concat(dfs)
dfs.head(2)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [33]:
#save interploated daily timeseries
dfs.to_parquet(outfile, compression='gzip')